In [1]:
import graphlab
import requests

ImportError: No module named graphlab

In [ ]:
discogs_frame = graphlab.SFrame('discogs.csv')

## Rating: 0 unrated, 1-5 rated

## Category: 0 = collection, 1 = wantlist, 2 = for sale

In [ ]:
discogs_frame.rename({'1':'userId', '63257':'releaseId', '0':'rating','0.1':'category'}) # Lost a row here renaming. Add it later

In [ ]:
graphlab.canvas.set_target('ipynb')
discogs_frame.show()

#### ~215,000  users

# Find the most occuring releases

In [ ]:
most_frequent_releases = discogs_frame.groupby(key_columns={'releaseId'}, operations={'appearance_count':
                                                           graphlab.aggregate.COUNT('releaseId')}).sort('appearance_count', ascending=False)

In [ ]:
most_frequent_releases

## Get the human readable names of these releases

In [ ]:
discogskey = open('discogskey.txt').readline() # Get my Discogs API token

In [ ]:
payload = {'token':discogskey}
def get_release_name(releaseId):
    response = requests.get('https://api.discogs.com/releases/' + str(releaseId), params=payload)
    print response.json()['artists'][0]['name'] + " " + response.json()['title']

In [ ]:
for releaseId in most_frequent_releases['releaseId'][0:10]:
    get_release_name(str(releaseId))

## Number of rated releases

In [ ]:
print "One star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==1]), ",d"))
print "Two star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==2]), ",d"))
print "Three star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==3]), ",d"))
print "Four star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==4]), ",d"))
print "Five star ratings: " + str(format(len(discogs_frame[discogs_frame['rating']==5]), ",d"))

In [ ]:
print "Unrated: " + str(format(len(discogs_frame[discogs_frame['rating']==0]), ",d"))

## Create a subset containing only collection/wantlist/for sale without the 3 star ratings or below

In [ ]:
no_star_frame = discogs_frame[discogs_frame['rating']==0]
four_or_five_star_frame = discogs_frame[discogs_frame['rating']>3]

In [ ]:
no_star_frame.show()

In [ ]:
no_four_five_star_frame = no_star_frame.append(four_or_five_star_frame)

In [ ]:
no_four_five_star_frame.show()

## Create an item similarity recommender

In [ ]:
train_data, test_data = graphlab.recommender.util.random_split_by_user(no_four_five_star_frame,
                                                                      user_id='userId',
                                                                      item_id='releaseId',
                                                                      max_num_users=None,
                                                                      random_seed=0)

In [ ]:
len(test_data)

In [ ]:
len(train_data)

In [ ]:
item_similarity_model = graphlab.item_similarity_recommender.create(train_data, 
                                                                   user_id='userId',
                                                                   item_id='releaseId',
                                                                   # don't use target= as '0' ratings exist
                                                                   similarity_type='jaccard', #cosine or pearson
                                                                   )